#MUSHROOM CLASSIFICATION - KAGGLE COMPETITION
so from now on im going to make a model for mushrrom classification to upveal my skills

In [ ]:
!pip install tqdm torchinfo

In [ ]:
# neccessary imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets

In [ ]:
# reading the csv file and playing around
data = pd.read_csv('train.csv')
data.head(10)

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
5,5,p,4.30,x,t,n,f,s,c,n,...,NaN,NaN,w,NaN,n,t,z,NaN,d,a
6,6,e,9.65,p,y,w,f,e,c,k,...,NaN,s,w,NaN,NaN,t,e,NaN,g,w
7,7,p,4.55,x,e,e,f,a,NaN,y,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
8,8,p,7.36,f,h,e,f,x,d,w,...,b,NaN,w,NaN,NaN,f,f,NaN,d,a
9,9,e,6.45,x,t,n,f,a,d,w,...,NaN,NaN,e,NaN,NaN,f,f,NaN,d,a


In [ ]:
data.value_counts()

,,,,,,,,,,,,,,,,,,,,,,count
id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,


In [ ]:
data.columns

Index(['id', 'class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season'],
      dtype='object')

In [ ]:
data['gill-spacing'].unique()

array(['c', nan, 'd', 'f', 'x', 'b', 'a', '3.61', '2.69', 'k', '4.8', 'e'],
      dtype=object)

In [ ]:
# Select numerical columns
numerical_cols = data.select_dtypes(include=['number']).columns

# Select categorical columns (excluding numerical types)
categorical_cols = data.select_dtypes(exclude=['number']).columns

# Print the selected columns
print("Numerical Columns:")
print(numerical_cols)

print("\nCategorical Columns:")
print(categorical_cols)

Numerical Columns:
Index(['id', 'cap-diameter', 'stem-height', 'stem-width'], dtype='object')

Categorical Columns:
Index(['class', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-root', 'stem-surface', 'stem-color', 'veil-type', 'veil-color',
       'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season'],
      dtype='object')


In [ ]:
# Filling NaN in numerical columns with 0
data[numerical_cols] = data[numerical_cols].fillna(0)

data[categorical_cols] = data[categorical_cols].fillna('Unknown')
(data)

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,Unknown,Unknown,w,Unknown,Unknown,f,f,Unknown,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,Unknown,y,o,Unknown,Unknown,t,z,Unknown,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,Unknown,s,n,Unknown,Unknown,f,f,Unknown,l,w
3,3,e,3.88,f,y,g,f,s,Unknown,g,...,Unknown,Unknown,w,Unknown,Unknown,f,f,Unknown,d,u
4,4,e,5.85,x,l,w,f,d,Unknown,w,...,Unknown,Unknown,w,Unknown,Unknown,f,f,Unknown,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139667,139667,e,2.57,x,h,p,f,a,c,o,...,Unknown,Unknown,o,Unknown,Unknown,f,f,Unknown,g,w
139668,139668,p,0.77,x,g,y,f,Unknown,Unknown,y,...,Unknown,Unknown,y,Unknown,Unknown,f,f,Unknown,d,a
139669,139669,e,11.86,s,Unknown,w,t,d,c,y,...,Unknown,Unknown,w,Unknown,Unknown,f,f,Unknown,d,u
139670,139670,e,5.93,x,s,n,f,x,c,w,...,Unknown,s,n,Unknown,Unknown,f,f,Unknown,l,w


In [ ]:
data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,Unknown,Unknown,w,Unknown,Unknown,f,f,Unknown,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,Unknown,y,o,Unknown,Unknown,t,z,Unknown,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,Unknown,s,n,Unknown,Unknown,f,f,Unknown,l,w
3,3,e,3.88,f,y,g,f,s,Unknown,g,...,Unknown,Unknown,w,Unknown,Unknown,f,f,Unknown,d,u
4,4,e,5.85,x,l,w,f,d,Unknown,w,...,Unknown,Unknown,w,Unknown,Unknown,f,f,Unknown,g,a


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
print(f"TRAIN DATA : \n{train_data}")
print(f"TEST DATA : \n{test_data}")

# so class is our target and rest all except id is our inputs

TRAIN DATA : 
            id class  cap-diameter cap-shape cap-surface cap-color  \
62602    62602     e          3.30         s           d         g   
40266    40266     p          8.73         o           t         o   
48274    48274     e          5.54         o     Unknown         g   
128525  128525     p          2.56         x           g         n   
66357    66357     p          2.59         x           g         n   
...        ...   ...           ...       ...         ...       ...   
110268  110268     p          1.40         x           g         n   
119879  119879     p          4.04         x           s         y   
103694  103694     e         11.12         x           h         n   
131932  131932     p          7.28         f           t         e   
121958  121958     e          1.63         f           w         g   

       does-bruise-or-bleed gill-attachment gill-spacing gill-color  ...  \
62602                     t               d            c          p  

In [ ]:
train_data['class'][10]

'e'

In [ ]:
stoi

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25,
 'unknown': 100}

In [ ]:
# lets make the dataset first
from sklearn.preprocessing import StandardScaler

stoi = {}
for i in range(26):
  stoi[chr(97 + i)] = i

stoi['Unknown'] = 0

def encode(x):
  try:
    return stoi[x]
  except:
    return stoi['Unknown'] # when it was returning x it was causing error

# 0 for p and 1 for e
def encode_class(x):
  if x == 'e':
    return 1
  else:
    return 0

class datasett(torch.utils.data.Dataset):
  def __init__(self, data):
    self.data = data

    self.targets = self.data['class'].map(encode_class)
    self.data = self.data.applymap(encode)
    self.df_except_one = self.data.drop(columns=['id', 'class'])

    self.scaler = StandardScaler()
    self.df_except_one = pd.DataFrame(self.scaler.fit_transform(self.df_except_one))


  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    inputs = self.df_except_one.iloc[idx]
    targets = self.targets.iloc[idx]
    z = inputs.values
    z = z.astype(np.float32)
    return torch.from_numpy(z), torch.tensor(targets)

train_dataset = datasett(train_data)
test_dataset = datasett(test_data)
train_dataset

<ipython-input-57-0e83a700f6f5>:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  self.data = self.data.applymap(encode)
<ipython-input-57-0e83a700f6f5>:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  self.data = self.data.applymap(encode)


In [ ]:
print(train_data.iloc[10].values)

[64494 'p' 7.14 's' 't' 'y' 'f' 'x' 'c' 'y' 5.39 14.5 'Unknown' 'Unknown'
 'y' 'Unknown' 'Unknown' 'f' 'f' 'Unknown' 'd' 'a']


In [ ]:
train_dataset[0]

(tensor([-10.,  18.,   3.,   6.,  19.,   3.,   2.,  15., -10., -10., -10., -10.,
           6., -10., -10.,   5.,   5., -10.,   3.,   0.]),
 tensor(1))

In [ ]:
train_dataset[0]

(tensor([ 0.0000,  0.3457, -0.8947, -1.4129,  2.1607, -0.4641,  0.4125, -0.2413,
          0.0000,  0.0000, -0.2574, -0.6751, -1.9733, -0.2306, -0.3602, -0.5602,
         -0.2507, -0.2884, -0.5292, -1.0045]),
 tensor(1))

In [ ]:
# lets make a dataloader for it
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True, drop_last=True)
train_dataloader

In [ ]:
next(iter(train_dataloader))

[tensor([[ 0.0000, -1.5777,  0.9710, -0.2437, -0.4628,  1.9372,  0.4125, -0.2413,
           0.0000,  0.0000, -0.2574,  0.2611, -1.9733, -0.2306, -0.3602, -0.5602,
          -0.2507, -0.2884, -0.5292, -1.0045],
         [ 0.0000,  0.9114, -1.2446,  0.4244, -0.4628, -0.8243, -1.0828,  0.0700,
           0.0000,  0.0000, -0.2574, -0.6751, -0.0614, -0.2306, -0.3602, -0.5602,
          -0.2507, -0.2884, -0.5292,  0.9732],
         [ 0.0000, -1.1252,  1.5540, -0.2437, -0.4628, -0.8243,  1.1601, -0.2413,
           0.0000,  0.0000, -0.2574,  0.2611, -0.7566, -0.2306, -0.3602, -0.5602,
          -0.2507, -0.2884,  0.7453, -1.0045],
         [ 0.0000,  0.9114,  0.8544, -0.2437,  2.1607,  0.9767, -1.0828,  1.1595,
           0.0000,  0.0000, -0.2574, -0.6751, -0.7566, -0.2306,  2.8738,  1.7851,
           1.9503, -0.2884, -0.5292, -1.0045],
         [ 0.0000,  0.9114, -0.7781, -0.2437,  2.1607, -0.3440, -1.0828,  0.8482,
           0.0000,  0.0000, -0.2574, -0.6751,  0.8077, -0.2306, -0.3602,  

In [ ]:
train_dataset[0][0].shape

torch.Size([20])

In [ ]:
import torch.nn.functional as F

z = next(iter(train_dataloader))
e = nn.Embedding(101, 10)
out = e(z[0].to(torch.long))
out.shape

torch.Size([32, 20, 10])

In [ ]:
# TODO : correct the dataset code and make it able to return tensors we will need to do a word emebdding logically not word but character
import torch.nn.functional as F

# MODEL : Most simplest one
class mushroom_model_v1(nn.Module):
  def __init__(self, in_features, out = 1):
    super().__init__()
    self.em = nn.Embedding(101, 10)
    self.l1 = nn.Linear(in_features=10, out_features=50)
    self.l2 = nn.Linear(in_features=50, out_features= 100)
    self.l3 = nn.Linear(in_features=100, out_features=50)
    self.l4 = nn.Linear(in_features=50, out_features=out)

    self.m = nn.Sequential(
        nn.Linear(out, 100),
        nn.ReLU(),
        nn.Linear(100, 200),
        nn.ReLU(),
        nn.Linear(200, 500),
        nn.ReLU(),
        nn.Linear(500, 100),
        nn.ReLU(),
        nn.Linear(100, 1),
        nn.Sigmoid()
    )

    self.flat = nn.Flatten(start_dim = 1)
    self.l5 = nn.Linear(in_features=20, out_features=10)
    self.l6 = nn.Linear(in_features=10, out_features=out)

  def forward(self, x):
    x = self.em(x)
    x = F.leaky_relu(self.l1(x))
    x = F.leaky_relu(self.l2(x))
    x = F.leaky_relu(self.l3(x))
    x = F.silu(self.l4(x))
    x = self.m(x)

    x = self.flat(x)
    x = F.silu(self.l5(x))
    x = F.silu(self.l6(x))
    # return x
    return F.softmax(x, dim = 1)


model_1 = mushroom_model_v1(in_features=20, out=1)
model_1

mushroom_model_v1(
  (em): Embedding(101, 10)
  (l1): Linear(in_features=10, out_features=50, bias=True)
  (l2): Linear(in_features=50, out_features=100, bias=True)
  (l3): Linear(in_features=100, out_features=50, bias=True)
  (l4): Linear(in_features=50, out_features=1, bias=True)
  (m): Sequential(
    (0): Linear(in_features=1, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=200, bias=True)
    (3): ReLU()
    (4): Linear(in_features=200, out_features=500, bias=True)
    (5): ReLU()
    (6): Linear(in_features=500, out_features=100, bias=True)
    (7): ReLU()
    (8): Linear(in_features=100, out_features=1, bias=True)
    (9): Sigmoid()
  )
  (flat): Flatten(start_dim=1, end_dim=-1)
  (l5): Linear(in_features=20, out_features=10, bias=True)
  (l6): Linear(in_features=10, out_features=1, bias=True)
)

In [ ]:
# TODO : correct the dataset code and make it able to return tensors we will need to do a word emebdding logically not word but character
import torch.nn.functional as F

# MODEL : Most simplest one
class mushroom_model_v1(nn.Module):
  def __init__(self, in_features, out = 1):
    super().__init__()
    # self.em = nn.Embedding(101, 10)
    self.l1 = nn.Linear(in_features=20, out_features=50)
    self.l2 = nn.Linear(in_features=50, out_features= 100)
    self.l3 = nn.Linear(in_features=100, out_features=50)
    self.l4 = nn.Linear(in_features=50, out_features=out)

    self.m = nn.Sequential(
        nn.Linear(out, 100),
        nn.ReLU(),
        nn.Linear(100, 200),
        nn.ReLU(),
        nn.Linear(200, 500),
        nn.ReLU(),
        nn.Linear(500, 100),
        nn.ReLU(),
        nn.Linear(100, 1),
        nn.Sigmoid()
    )

    # self.flat = nn.Flatten(start_dim = 1)
    self.l5 = nn.Linear(in_features=1, out_features=10)
    self.l6 = nn.Linear(in_features=10, out_features=out)

  def forward(self, x):
    # x = self.em(x)
    x = F.leaky_relu(self.l1(x))
    x = F.dropout(self.l2(x))
    x = F.leaky_relu(self.l3(x))
    x = F.dropout(self.l4(x))
    x = self.m(x)

    x = F.dropout(self.l5(x))
    x = F.silu(self.l6(x))
    # return x
    return F.softmax(x, dim = 1)


model_1 = mushroom_model_v1(in_features=20, out=1)
model_1

mushroom_model_v1(
  (l1): Linear(in_features=20, out_features=50, bias=True)
  (l2): Linear(in_features=50, out_features=100, bias=True)
  (l3): Linear(in_features=100, out_features=50, bias=True)
  (l4): Linear(in_features=50, out_features=1, bias=True)
  (m): Sequential(
    (0): Linear(in_features=1, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=200, bias=True)
    (3): ReLU()
    (4): Linear(in_features=200, out_features=500, bias=True)
    (5): ReLU()
    (6): Linear(in_features=500, out_features=100, bias=True)
    (7): ReLU()
    (8): Linear(in_features=100, out_features=1, bias=True)
    (9): Sigmoid()
  )
  (l5): Linear(in_features=1, out_features=10, bias=True)
  (l6): Linear(in_features=10, out_features=1, bias=True)
)

In [ ]:
out = model_1(z[0])
out.shape

torch.Size([32, 1])

In [ ]:
# torch.flatten(out,start_dim = 1).shape

torch.Size([32, 20])

In [ ]:
# making loss function nad optimizers

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=model_1.parameters(), lr=0.001)

In [ ]:
# this time we will also measure accuracy since in past few projects the outputs were too complexed for accuracy to be computed
from sklearn.metrics import accuracy_score
NUM_EPOCHS = 20

train_list = []
test_list = []
train_acc_list = []
test_acc_list = []

for epoch in tqdm(range(NUM_EPOCHS)):
  train_loss = 0
  train_acc = 0
  model_1.train()
  for inputs, targets in train_dataloader:
    pred = model_1(inputs) #.to(torch.long))
    loss = loss_fn(pred.squeeze(), targets.float())
    acc = accuracy_score(targets.detach().numpy(), pred.squeeze().detach().numpy())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
    train_acc += acc.item()

  model_1.eval()
  with torch.inference_mode():
    test_loss = 0
    test_acc = 0
    for inputs, targets in test_dataloader:
      pred = model_1(inputs) #.to(torch.long))
      loss = loss_fn(pred.squeeze(), targets.float())
      acc = accuracy_score(targets.detach().numpy(), pred.squeeze().detach().numpy())
      test_loss += loss.item()
      test_acc += acc.item()


  test_loss /= len(test_dataloader)
  train_loss /= len(train_dataloader)
  test_acc /= len(test_dataloader)
  train_acc /= len(train_dataloader)
  print(f"Epoch : {epoch} | Train Loss : {train_loss}, Train acc : {train_acc:.2f}% | Test Loss : {test_loss}, Test acc : {test_acc:.2f}%")


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch : 0 | Train Loss : 0.8614614705407287, Train acc : 0.45% | Test Loss : 0.861294649000656, Test acc : 0.45%
Epoch : 1 | Train Loss : 0.8614103246242442, Train acc : 0.45% | Test Loss : 0.8612230275312938, Test acc : 0.45%
Epoch : 2 | Train Loss : 0.8614410114854135, Train acc : 0.45% | Test Loss : 0.861151408429725, Test acc : 0.45%
Epoch : 3 | Train Loss : 0.8614307834356779, Train acc : 0.45% | Test Loss : 0.8611752795663109, Test acc : 0.45%
Epoch : 4 | Train Loss : 0.861430781679731, Train acc : 0.45% | Test Loss : 0.8611752813876905, Test acc : 0.45%


KeyboardInterrupt: 

In [ ]:
"""
MODEL 1 = SIMPLE 2 LINEAR LAYERS WITH SOFTMAX
Epoch : 0 | Train Loss : 0.8620516618998013, Train acc : 0.45% | Test Loss : 0.8606687108252911, Test acc : 0.45%
Epoch : 1 | Train Loss : 0.8620801363284604, Train acc : 0.45% | Test Loss : 0.8604692443254146, Test acc : 0.45%
Epoch : 2 | Train Loss : 0.8620801346720215, Train acc : 0.45% | Test Loss : 0.8603695105364982, Test acc : 0.45%
Epoch : 3 | Train Loss : 0.8620943695103117, Train acc : 0.45% | Test Loss : 0.860469243818141, Test acc : 0.45%
Epoch : 4 | Train Loss : 0.8621086081774196, Train acc : 0.45% | Test Loss : 0.8603362626217781, Test acc : 0.45%''



"""

(tensor([1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 1, 1, 0, 1, 0, 0, 0]),
 tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]], grad_fn=<SoftmaxBackward0>))

In [ ]:
# F.softmax(model_1(inputs.float()), dim = 1)

<ipython-input-56-c72851159c80>:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.l2(self.l1(x)))


tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SoftmaxBackward0>)

# trying ML

In [1]:
!gdown 10ObP-bQeebGtsd23Nd347yuWiuJsDB1f
!gdown 1tASigMmrBP0FK_WdtXZLq5iFHHRbDoJu

Downloading...
From (original): https://drive.google.com/uc?id=10ObP-bQeebGtsd23Nd347yuWiuJsDB1f
From (redirected): https://drive.google.com/uc?id=10ObP-bQeebGtsd23Nd347yuWiuJsDB1f&confirm=t&uuid=d325a5b7-119b-4f19-8d08-3f43243167ca
To: /content/train.csv
100% 168M/168M [00:06<00:00, 25.9MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1tASigMmrBP0FK_WdtXZLq5iFHHRbDoJu
From (redirected): https://drive.google.com/uc?id=1tASigMmrBP0FK_WdtXZLq5iFHHRbDoJu&confirm=t&uuid=39101e4b-0d09-4ccc-9b3a-6053a006722f
To: /content/test.csv
100% 109M/109M [00:02<00:00, 40.9MB/s]


In [13]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv("train.csv")

stoi = {}
for i in range(26):
  stoi[chr(97 + i)] = i

stoi['Unknown'] = 0

def encode(x):
  try:
    return stoi[x]
  except:
    return stoi['Unknown'] # when it was returning x it was causing error

# 0 for p and 1 for e
def encode_class(x):
  if x == 'e':
    return 1
  else:
    return 0


# DATA is the varname for our data
y = data['class'].map(encode_class)
x = data.map(encode)
X = x.drop(columns=['id', 'class', 'veil-type'	])

scaler = StandardScaler()
# print(X)
X = pd.DataFrame(scaler.fit_transform(X))


# Split dataset into training set and test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) # 70% training and 30% test

# Create Decision Tree classifer object
# clf = DecisionTreeClassifier()
# clf = RandomForestClassifier()
# rf.fit(X_train, y_train)

# from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier,ExtraTreesClassifier
# r1 = RandomForestClassifier()
# r2 = RandomForestClassifier()
# r3 = RandomForestClassifier()
# r4 = RandomForestClassifier()

# clf = StackingClassifier(estimators=[('r1', r1), ('r2', r2), ('r3', r3), ('r4', r4)])

# c1 = ExtraTreesClassifier()
# c2 = ExtraTreesClassifier()
# clf = StackingClassifier(estimators=[('r1', c2), ('r2', c1)])
# from sklearn.neural_network import MLPClassifier

# clf = MLPClassifier()


In [14]:
xgb_params = {
    'n_estimators': 5000,
    'alpha': 0.0002,
    'subsample': 0.60,
    'colsample_bytree': 0.4,
    'max_depth': 13,
    'min_child_weight': 12,
    'learning_rate': 0.04,
    'gamma': 5.6e-08,
    'tree_method': 'hist',
    'device': 'cuda'
}

In [15]:
from joblib import parallel_backend
from sklearn.ensemble import ExtraTreesClassifier, HistGradientBoostingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

# from sklearn.svm import SVC
from xgboost import XGBClassifier
# model = SVC(gamma=0.7, C=1e6)



with parallel_backend('threading', n_jobs=160):
    # Your scikit-learn code here
    clf = XGBClassifier(**xgb_params)

    # Train Decision Tree Classifer
    clf = clf.fit(X,y)

    #Predict the response for test dataset
    # y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [8]:
from sklearn.metrics import matthews_corrcoef

matthews_corrcoef(y_test, y_pred)


0.9816717307823549

In [ ]:
d = pd.DataFrame(y_pred)
new_df = data['id']
new_df = pd.concat([new_df,d], axis = 1)
new_df
# new_df.to_csv('submission.csv', index = False)

,id,0
0,0,0.0
1,1,1.0
2,2,1.0
3,3,1.0
4,4,0.0
...,...,...
139667,139667,NaN
139668,139668,NaN
139669,139669,NaN
139670,139670,NaN


#submis

In [16]:
test_data = pd.read_csv('test.csv')

# Select numerical columns
numerical_cols = test_data.select_dtypes(include=['number']).columns

# Select categorical columns (excluding numerical types)
categorical_cols = test_data.select_dtypes(exclude=['number']).columns


# Filling NaN in numerical columns with 0
test_data[numerical_cols] = test_data[numerical_cols].fillna(0)

test_data[categorical_cols] = test_data[categorical_cols].fillna('Unknown')


# DATA is the varname for our data
# y = test_data['class'].map(encode_class)   # SINCE WE DONT HAVE THE CLASS KNOWN FOR IT
x = test_data.map(encode)
X = x.drop(columns=['id', 'veil-type'	])


scaler = StandardScaler()
# print(X)
X = pd.DataFrame(scaler.fit_transform(X))

y_pred = clf.predict(X)
# y_pred = automl.predict(X)

y_pred

array([0, 0, 0, ..., 0, 1, 1])

In [28]:
len(y_pred)

2077964

In [17]:
def decode_class(x):
  if x == 1:
    return 'e'
  else:
    return 'p'

d = pd.DataFrame(y_pred).map(decode_class)
new_df = test_data['id']
new_df = pd.concat([new_df,d], axis = 1)

new_df.rename(columns={'id': 'id', 0: 'class'}, inplace=True)

new_df.to_csv('submission.csv', index = False)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()

#new ways

In [4]:
!pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 7.6 MB/s eta 0:00:00


In [21]:
from flaml import AutoML
from flaml.automl.model import LGBMEstimator

automl = AutoML()

automl.add_learner("mylgbm", LGBMEstimator)
automl.fit(
    X_train,
    y_train,
    task="classification",
    metric='accuracy',
    estimator_list=["mylgbm"],
    time_budget=60,
)

[flaml.automl.logger: 08-24 19:59:57] {1680} INFO - task = classification
[flaml.automl.logger: 08-24 19:59:57] {1691} INFO - Evaluation method: holdout
[flaml.automl.logger: 08-24 20:00:01] {1789} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 08-24 20:00:01] {1901} INFO - List of ML learners in AutoML Run: ['mylgbm']
[flaml.automl.logger: 08-24 20:00:01] {2219} INFO - iteration 0, current learner mylgbm
[flaml.automl.logger: 08-24 20:00:01] {2345} INFO - Estimated sufficient time budget=257638s. Estimated necessary time budget=258s.
[flaml.automl.logger: 08-24 20:00:01] {2392} INFO -  at 13.4s,	estimator mylgbm's best error=0.3727,	best estimator mylgbm's best error=0.3727
[flaml.automl.logger: 08-24 20:00:01] {2219} INFO - iteration 1, current learner mylgbm
[flaml.automl.logger: 08-24 20:00:01] {2392} INFO -  at 13.5s,	estimator mylgbm's best error=0.3727,	best estimator mylgbm's best error=0.3727
[flaml.automl.logger: 08-24 20:00:01] {2219} INFO - iteration 2, cu

In [22]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))


Best ML leaner: mylgbm
Best hyperparmeter config: {'n_estimators': 443, 'num_leaves': 353, 'min_child_samples': 3, 'learning_rate': 0.34880620114884875, 'log_max_bin': 6, 'colsample_bytree': 0.40062825735955104, 'reg_alpha': 0.0009765625, 'reg_lambda': 0.002677090392149446}
Best accuracy on validation data: 0.9892
Training duration of best run: 264.2 s


In [23]:
y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)

In [24]:
from flaml.ml import sklearn_metric_loss_score
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred, y_test))
# print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test))

"""
accuracy = 0.990645760167001
log_loss = 0.04201363489924512
"""

accuracy = 0.9900073148508584
log_loss = 0.045519357543035995


'\naccuracy = 0.990645760167001\nlog_loss = 0.04201363489924512\n\n'

In [20]:
from sklearn.metrics import classification_report
cr = classification_report(y_test,y_pred)
print(cr)


              precision    recall  f1-score   support

           0       0.99      0.99      0.99    511799
           1       0.99      0.99      0.99    423285

    accuracy                           0.99    935084
   macro avg       0.99      0.99      0.99    935084
weighted avg       0.99      0.99      0.99    935084



#sklearn automl

In [6]:
!pip3 install auto-sklearn

  Using cached auto-sklearn-0.15.0.tar.gz (6.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached scikit-learn-0.24.2.tar.gz (7.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [5]:
import autosklearn.classification
from sklearn.metrics import accuracy_score, classification_report

# Initialize Auto-sklearn classifier
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600,  # Total time in seconds (e.g., 1 hour)
    per_run_time_limit=300,        # Time limit for each model (e.g., 5 minutes)
    n_jobs=-1,                     # Use all available cores
    ensemble_size=50,              # Size of the ensemble built by Auto-sklearn
    initial_configurations_via_metalearning=25,
    seed=42
)

# Fit the model
automl.fit(X_train, y_train, dataset_name='mushroom_classification')

# Print the final ensemble constructed by auto-sklearn
print(automl.show_models())

# Predict on the test set
y_pred = automl.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

print('Classification Report:')
print(classification_report(y_test, y_pred))


ModuleNotFoundError: No module named 'autosklearn'

In [ ]:
from sklearn.metrics import matthews_corrcoef

matthews_corrcoef(y_test, y_pred)


# more of other ways

In [ ]:
import cupy as cp
import cudf
from cuml.preprocessing import TargetEncoder
from cuml.svm import SVC
model = SVC(gamma=0.7, C=1e6)


In [ ]:
model = SVC(gamma=0.7, C=1e6)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'MCC: {cp.corrcoef(y_test, y_pred)[0,1]:.5f}')